In [8]:
import numpy as np 
import json
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [9]:
with open("intents.json") as file:
  data=json.load(file)
training_sentences = []
training_labels = []
labels = []
response= []
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    response.append(intent["responses"])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)
labels

['greeting',
 'goodbye',
 'thanks',
 'about',
 'name',
 'help',
 'createaccount',
 'complaint']

In [10]:
label_encoder=LabelEncoder()
label_encoder.fit(training_labels)
training_labels=label_encoder.transform(training_labels)

TOKENIZATION


In [11]:
vocab_size=1000
embedding_dim=16
max_len=60
oov_token="<00V>"

tokenizer =Tokenizer(num_words=vocab_size,oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded_sequences=pad_sequences(sequences,truncating='post',maxlen=max_len)

TRAINING NEURAL NETWORK


In [12]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy",optimizer='Adam',metrics=['accuracy'])

model.summary()
epochs=500
history=model.fit(padded_sequences,np.array(training_labels),epochs=epochs)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 60, 16)            16000     
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_5 (Dense)             (None, 1024)              17408     
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                      

SAVING THE MODEL 


In [13]:
#saving the trained model
model.save("chat_bot_model")

import pickle

#save the fitted tokenizer 
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

#save the fitted label encoder
with open('label_encoder.pickle','wb') as ecn_file:
    pickle.dump(label_encoder,ecn_file,protocol=pickle.HIGHEST_PROTOCOL)  

INFO:tensorflow:Assets written to: chat_bot_model/assets


INFO:tensorflow:Assets written to: chat_bot_model/assets
